In [1]:
!pip install -U -q bitsandbytes
# !pip install transformers==4.36.2
!pip install -U -q peft accelerate trl sentencepiece transformers tokenizers
!pip install -q datasets==2.16.0



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

In [3]:
# base model from huggingFace or path to model
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "Enlighten_Instruct-alan-v0.2"

test_path='/opt/app-root/src/Enlighten-Instruct/Dataset/TestData.csv'
train_path='/opt/app-root/src/Enlighten-Instruct/Dataset/TrainData.csv'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [5]:
from transformers import LlamaTokenizer
tokenizer_ = LlamaTokenizer.from_pretrained("cognitivecomputations/dolphin-llama2-7b")
# tokens = tokenizer_.tokenize(dataset.to_pandas().to_string())
# len(tokens)

In [6]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", 
                model=new_model, 
                tokenizer=tokenizer, 
                max_length=400)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def build_prompt(question):
  prompt=f"<s>[INST]@Enlighten. {question} [/INST]"
  return prompt

In [8]:
question = "what is the pwoer outage event?"
prompt = build_prompt(question)
result = pipe(prompt)

print(result[0]['generated_text'])

<s>[INST]@Enlighten. what is the pwoer outage event? [/INST] I believe you may be asking about power outages, which are disruptions in the electrical power supply to homes, businesses, or other consumers. Power outages can be caused by various reasons, including severe weather conditions, equipment failures, accidents, or intentional actions. They can last from a few minutes to several hours or even days, depending on the cause and the efforts to restore power. Power outages can have significant impacts on daily life, including disrupting communication, transportation, and essential services. Utilities and power companies work to minimize the duration and frequency of power outages and to restore power as quickly and safely as possible.


In [ ]:
df_test=pd.read_csv(test_path)

questionCounter=0
correct=0
promptEnding = "[/INST]"

# this must be >= 2
fail_limit=10

# chain of thought activator, model might run out of output tokens
USE_COT=True

#this comes before the question
testGuide='Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d \n'

for index, row in df_test.iterrows():
    print("#############################")
    questionCounter = questionCounter + 1

    #chain of thought activator
    if USE_COT:
        chainOfThoughtActivator='\nfirst think step by step\n'
    else:
        chainOfThoughtActivator='\n'

    #build the prompt
    question=testGuide + row['Question'] + '\na)' + row['a'] + '\nb)' + row['b'] + '\nc)' + row['c'] + '\nd)' + row['d'] + chainOfThoughtActivator
    print(question)

    #true answer
    truth=row['Answer']

    #use a loop, if llm stopped before reaching the answer. ask again
    index=-1
    failCounter=0
    while(index==-1):

        #build the prompt
        prompt = build_prompt(question)

        #generate answer
        result = pipe(prompt)
        llmAnswer = result[0]['generated_text']

        #remove our prompt from it
        index = llmAnswer.find(promptEnding)
        llmAnswer = llmAnswer[len(promptEnding)+index:]

        print("LLM Answer:")
        print(llmAnswer)

        #remove spaces
        llmAnswer=llmAnswer.replace(' ','')

        #find the option in response
        index = llmAnswer.find('Answer:')

        #edge case - llm stoped at the worst time
        if(index+len('Answer:')==len(llmAnswer)):
            index=-1

        #update question for the next try. remove chain of thought
        question=testGuide + row['Question'] + '\na)' + row['a'] + '\nb)' + row['b'] + '\nc)' + row['c'] + '\nd)' + row['d']

        #Don't get stock on a question
        failCounter=failCounter+1
        if failCounter==fail_limit:
            break

    if failCounter==fail_limit:
        continue

    #find and match the option
    next_char = llmAnswer[index+len('Answer:'):][0]
    if next_char in truth:
        correct=correct+1
        print('correct')
    else:
        print('wrong')

    #update accuracy
    accuracy=correct/questionCounter
    print(f"Progress: {questionCounter/len(df_test)}")
    print(f"Accuracy: {accuracy}")




#############################
Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d 
What does the BeastAttack class do in the Unity project?
a)Manages player movement
b)Generates sounds
c)Manages the beast attack behavior and player hit behavior
d)Controls the camera
first think step by step

LLM Answer:
 The BeastAttack class in a Unity project is most likely responsible for managing the beast's attack behavior and player hit behavior. This includes determining when the beast attacks, handling the animation and effects of the attack, and applying damage to the player when they are hit. Therefore, the answer is:

Answer:c
correct
Progress: 0.0037174721189591076
Accuracy: 1.0
#############################
Answer the following question, at the end of your response write the answer like this: Answer:a or Answer:b or Answer:c or Answer:d 
What does the gameObject that uses the BeastAttack.cs script need to have 